In [5]:

import matplotlib.pyplot as plt
import argparse
import time
import pandas as pd
import pickle
import joblib
import numpy as np
from pathlib import Path
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score


import keras
import tensorflow as tf

sys.path.insert(0, '../..') #Pour inclure le dossier parent à report
%matplotlib inline

from src.generators.generator_image import ImageGenerator
from src.models.models_image import *

BATCH_SIZE = 32
EPOCHS_TEXT= 30
EPOCHS_IMAGE = 30
EPOCHS_FUSION = 50
NUM_FOLDS = 3
NUM_TARGETS = 84916
TARGET_SHAPE = [50, 50, 3]
TEST_SPLIT= .16
RANDOM_STATE = 123

index_train, index_test = train_test_split(np.arange(NUM_TARGETS), test_size=TEST_SPLIT, random_state=RANDOM_STATE)

model_image = ModelImage_ResNet50(
    suffix="_224",
    num_folds=NUM_FOLDS,
    epochs = EPOCHS_IMAGE,
    batch_size=BATCH_SIZE,
    load=True,
    save=True,
    report=True,
    summary=True,
    target_shape=TARGET_SHAPE,
    random_state=RANDOM_STATE,
)
   
    
image_generator = ImageGenerator(
    root_dir="data/raw/images/image_train/",
    csv_texts="data/raw/X_train_update.csv",
    csv_labels="data/raw/Y_train_CVw08PX.csv",
    target_shape=TARGET_SHAPE,
    samples=NUM_TARGETS,
    batch_size=BATCH_SIZE,
    preprocessor=model_image.get_preprocessor(),
    encoder_fitted=False,
    preprocessing_function=model_image.get_preprocess_input()

)
train_image_generator, test_image_generator = image_generator.split(split_indexes=[index_train, index_test], is_batch=False, )
print("train_targets : " , train_image_generator.targets)
print("test_targets : " , test_image_generator.targets)
   


Nombre d'images traitées : 84916
Nombre de targets traitées : 84916
train_targets :  [23 13 21 ...  8  9 23]
test_targets :  [22  0 12 ... 25  9 23]


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [6]:

#rc\models\image\image_MobileNetV2_224\{saved_model.pbtxt|saved_model.pb}
model_image.fit(
    train_data=train_image_generator,
    validation_data=test_image_generator,
    class_weight=image_generator.class_weight,
    crossval=False
)



Current fold : src\models\image\image_ResNet50_224
model loaded from src\models\image\image_ResNet50_224
Found 71329 validated image filenames belonging to 27 classes.
Found 13587 validated image filenames belonging to 27 classes.
Found 13587 validated image filenames belonging to 27 classes.
425/425 [==============================] - 89s 206ms/step
[ 8 10  8 ...  8  8  8]
[22  0 12 ... 25  9 23]
